# Ultimate Matrix
This is originally authored by *the man the myth the legend* `Hank`  
Later adapted by `Sam` to add some Jacobian stuff

# Changes from Hanks's version
 - Added `should_simplify = False` to the tMatrix constructor, so trig identites aren't substituted automatically (you can set this to true if you are a monster)
 - Added `jacobian_god_function`
    - The first parameter is a list of transforms **the first must be the identify transform (ie T_0_from_0)**. Contains n entries
    - The second is a list of booleans indicating whether each joint is prismatic (starts from the first joint). Contains n-1 entries.

## The Setup & Classes

In [9]:
# This trick means that if all the cells are reran, this cell is skipped
# %%capture ALREADY_RAN_PIP
# try:
#     ALREADY_RAN_PIP
# except:
#     !pip install sympy

from math import sin, cos, pi
import numpy as np
from typing import List
from numpy.lib.arraysetops import isin
from sympy import Matrix, Number, factor, trigsimp, simplify, sympify, N, nsimplify, pretty, Eq, nsolve, eye
from sympy.calculus.singularities import is_increasing
from sympy.solvers.diophantine.diophantine import InhomogeneousTernaryQuadratic
from sympy.solvers.solveset import nonlinsolve

In [15]:
def round_expr(expr, num_digits):
    return expr.xreplace({n : round(n, num_digits) for n in expr.atoms(Number)})

class tMatrix():
    """transformation matrix generator"""
    def __init__(self, theta, d, alpha, a, print_precision = 3, should_simplify = False) -> None:
        self.precision = print_precision
        self.my_symbles = []
        theta_mat: Matrix = self.get_theta_mat(theta,d)
        alpha_mat: Matrix = self.get_alpha_mat(alpha, a)
        self.my_T: Matrix = theta_mat * alpha_mat
        self.should_simplify = should_simplify

    def __str__(self) -> str:
        my_str = ""
        strings = []
        padding = 5
        min_col_width = 0
        col_widths = []
        print_by_row = False
        print_by_row_thresh = 150
        for i in range(4):
            for j in range(4):
                m = self.my_T[i,j]
                if self.should_simplify:
                    m = simplify(m)
                m = factor(m)
                m = nsimplify(m, rational = True)
                m = round_expr(m, self.precision)
                m = pretty(m)
                #m = N(m, 4)
                this_str = f"{m}"
                strings.append(this_str)
                
        max_string_num = 0
        for i in range(4):
            for j in range(4):
                this_str  = strings[j * 4 + i]
                if len(this_str) > max_string_num:
                    max_string_num = len(this_str)
            if max_string_num < min_col_width:
                max_string_num = min_col_width
            col_widths.append(max_string_num+padding)
            max_string_num = 0

        

        for i in range(4):
            for j in range(4):
                this_str  += strings[i * 4 + j]
            if len(this_str) > max_string_num:
                max_string_num = len(this_str) + padding * 4
            this_str = ""

        if max_string_num > print_by_row_thresh:
            print_by_row = True
            max_string_num = 50

        if print_by_row:
            my_str += "_"*int(max_string_num/2 - 4) + "_uMatrix" + "_"*int(max_string_num/2 - 4) + "_"*10
        else:
            my_str += "_"*int(max_string_num/2 - 4) + "_uMatrix" + "_"*int(max_string_num/2 - 4)
        my_str += "\n\n"
        
        for i in range(4):
            for j in range(4):
                if print_by_row:
                    this_str  = strings[i * 4 + j]
                    my_str += f"row {i}, col{j}: {this_str}\n"
                else:
                    this_str  = strings[i * 4 + j]
                    len_diff = col_widths[j] - len(this_str)
                    my_str += this_str
                    for k in range(len_diff):
                        my_str += " "
                    # my_str += "\t"
            my_str += "\n"

        if print_by_row:
            my_str += "_" * max_string_num + "_"*10
        else:
            my_str += "_" * max_string_num

        my_str += "\n"

        return my_str

    
    def __mul__(self, other: 'tMatrix') -> 'tMatrix':
        retval = tMatrix(0,0,0,0)
        retval.my_T = self.my_T * other.my_T
        retval.my_symbles = self.my_symbles + other.my_symbles
        return retval
    
    def get_theta_mat(self, theta, d) -> Matrix:
        if isinstance(theta, str):
            theta_r = theta
        else:
            theta_r = theta * pi / 180

        e00 = sympify(f"cos({theta_r})")
        for i in e00.free_symbols:
            self.my_symbles.append(i)
        e01 = sympify(f"-sin({theta_r})")
        e10 = sympify(f"sin({theta_r})")
        e11 = sympify(f"cos({theta_r})")
        e02 = sympify("0")
        e03 = sympify("0")
        e12 = sympify("0")
        e13 = sympify("0")
        e20 = sympify("0")
        e21 = sympify("0")
        e22 = sympify("1")
        e23 = sympify(f"{d}")
        for i in e23.free_symbols:
            self.my_symbles.append(i)
        e30 = sympify("0")
        e31 = sympify("0")
        e32 = sympify("0")
        e33 = sympify("1")
        my_mat = [[e00,e01,e02,e03],[e10,e11,e12,e13],[e20,e21,e22,e23],[e30,e31,e32,e33]]
        return Matrix(my_mat)

    def get_alpha_mat(self,alpha,a):
        if isinstance(alpha, str):
            alpha_r = alpha
        else:
            alpha_r = alpha * pi / 180
        e11 = sympify(f"cos({alpha_r})")
        for i in e11.free_symbols:
            self.my_symbles.append(i)
        e12 = sympify(f"-sin({alpha_r})")
        e21 = sympify(f"sin({alpha_r})")
        e22 = sympify(f"cos({alpha_r})")


        e00 = sympify("1")
        e01 = sympify("0")
        e02 = sympify("0")
        
        e03 = sympify(f"{a}")
        for i in e11.free_symbols:
            self.my_symbles.append(i)
        e10 = sympify("0")
        
        e13 = sympify("0")
        e20 = sympify("0")

        e23 = sympify("0")
        e30 = sympify("0")
        e31 = sympify("0")
        e32 = sympify("0")
        e33 = sympify("1")
        test = [[e00,e01,e02,e03],[e10,e11,e12,e13],[e20,e21,e22,e23],[e30,e31,e32,e33]]
        return Matrix(test)

    def ik(self, target) -> dict:
        eqs = []
        syms = []
        init_guess = []
        for i in range(4):
            for j in range(4):
                # print(f"{i} {j} :{self.my_T[i, j].is_real}")
                if self.my_T[i, j].is_real:
                    pass
                else:
                    eq = Eq(self.my_T[i, j], target[i][j])
                    a = self.my_T[i, j].free_symbols
                    b = []
                    for k in a:
                        b.append(k)
                    syms.append(b)
                    # print(eq)
                    eqs.append(eq)
                    init_guess.append(0.5)
        # print(eqs[2])
        # print(self.my_symbles)
        num_syms = len(self.my_symbles)
        init_guess = []
        for i in range(num_syms):
            init_guess.append(0.5)
        results = nsolve(eqs, self.my_symbles, init_guess, verify = False)
        retval = {}
        for i, result in enumerate(results):
            retval[self.my_symbles[i]] = result
        return retval

In [13]:
def jacobian_god_function(t_matrices_list: List[tMatrix], is_prismatic_list: List[bool]) -> None:
    T_n_from_0 = eye(4)
    for i in range(len(t_matrices_list)):
        T_n_from_0 = T_n_from_0 * t_matrices_list[i].my_T

    P_n_from_0 = Matrix([T_n_from_0.row(0).col(3), T_n_from_0.row(1).col(3), T_n_from_0.row(2).col(3)])

    cols = []

    for i, t_matrix_n_from_0 in enumerate(t_matrices_list[:-1]):
        T_i_from_0 = eye(4)
        for j in range(i+1):
            T_i_from_0 = T_i_from_0 * t_matrices_list[j].my_T

        Z_i_from_0 = Matrix([T_i_from_0.row(0).col(2), T_i_from_0.row(1).col(2), T_i_from_0.row(2).col(2)])

        P_i_from_0 = Matrix([T_i_from_0.row(0).col(3), T_i_from_0.row(1).col(3), T_i_from_0.row(2).col(3)])
        r_i_from_0 = P_n_from_0 - P_i_from_0

        z_cross_r = Z_i_from_0.cross(r_i_from_0)

        if is_prismatic_list[i]:
            col_i = [Z_i_from_0.row(0).col(0), Z_i_from_0.row(1).col(0), Z_i_from_0.row(2).col(0), 0, 0, 0]

        else:
            col_i = [z_cross_r.row(0), z_cross_r.row(1), z_cross_r.row(2), Z_i_from_0.row(0), Z_i_from_0.row(1), Z_i_from_0.row(2)]

        if True:  # for debug
            print('i = ', i, '_' * 90)
            print('T   =', nsimplify(T_i_from_0, rational=True, tolerance=1e-10))
            print('Z   =', nsimplify(Z_i_from_0, rational=True, tolerance=1e-10))
            print('P   =', nsimplify(P_i_from_0, rational=True, tolerance=1e-10))
            print('r   =', nsimplify(r_i_from_0, rational=True, tolerance=1e-10))
            print('zxr =', nsimplify(z_cross_r, rational=True, tolerance=1e-10))
            print('col =', nsimplify(Matrix(col_i), rational=True, tolerance=1e-10))
            print('_' * 100)


        cols.append(col_i)

    jacobian = Matrix(cols).T
    jacobian = nsimplify(jacobian, rational = True)
    jacobian = round_expr(jacobian, 4)
    jacobian = factor(jacobian)

    
    print('=' * 100)
    print('JACOBIAN BELOW')
    print('=' * 100)

    for i in range(jacobian.shape[0]):
        line_parts = []
        for j in range(jacobian.shape[1]):
            m = jacobian[i,j]
            m = simplify(m)
            line_parts.append(pretty(m).ljust(50))
        print(' | '.join(line_parts))

## Examples

In [14]:
print("2019 Exam Q2b")
t0 = tMatrix(0, 0, 0, 0)
t1 = tMatrix("theta1", 1, 90, 0)
t2 = tMatrix("theta2", 0,0,1)
t3 = tMatrix("theta3", 0,0,0)
# t4 = tMatrix(0,0,0,"L3")
overall = t1*t2*t3
print(overall)

jacobian = jacobian_god_function([t0, t1, t2, t3], (False, False, False))

2019 Exam Q2b
____________________________________________________________________uMatrix___________________________________________________________________

-sin(θ₂)⋅sin(θ₃)⋅cos(θ₁) + cos(θ₁)⋅cos(θ₂)⋅cos(θ₃)     -sin(θ₂)⋅cos(θ₁)⋅cos(θ₃) - sin(θ₃)⋅cos(θ₁)⋅cos(θ₂)     sin(θ₁)      cos(θ₁)⋅cos(θ₂)     
-sin(θ₁)⋅sin(θ₂)⋅sin(θ₃) + sin(θ₁)⋅cos(θ₂)⋅cos(θ₃)     -sin(θ₁)⋅sin(θ₂)⋅cos(θ₃) - sin(θ₁)⋅sin(θ₃)⋅cos(θ₂)     -cos(θ₁)     sin(θ₁)⋅cos(θ₂)     
sin(θ₂)⋅cos(θ₃) + sin(θ₃)⋅cos(θ₂)                      -sin(θ₂)⋅sin(θ₃) + cos(θ₂)⋅cos(θ₃)                     0.0          sin(θ₂) + 1         
0                                                      0                                                      0            1                   
_______________________________________________________________________________________________________________________________________________

i =  0 __________________________________________________________________________________________
T   = Matrix([[1, 0, 0,

In [ ]:
print("2020 Exam")
t1 = tMatrix(45, 0, 0, 250)
t2 = tMatrix(-45, 0, 0, 400)
t3 = tMatrix(0, 100, 0, 0)
t4 = tMatrix(45, 0, 0, 0)
overall = t1*t2*t3*t4
print(overall)

2020 Exam
_________________uMatrix________________

0.707     -0.707     0     576.777     
0.707     0.707      0     176.777     
0         0          1     100         
0         0          0     1           
________________________________________



In [ ]:
print("Jacobian Homework Q2")
t0 = tMatrix(0, 0, 0, 0)
t1 = tMatrix(0, 'd1', 90, 0)
t2 = tMatrix(90, 'd2',-90,0)
t3 = tMatrix("theta3", 0,0,1)

jacobian = jacobian_god_function([t0, t1, t2, t3], (True, True, False)) # note True for the first two joints as they are prismatic

Jacobian Homework Q2
i =  0 __________________________________________________________________________________________
T   = Matrix([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
Z   = Matrix([[0], [0], [1]])
P   = Matrix([[0], [0], [0]])
r   = Matrix([[0], [-d2 + sin(theta3)], [d1 + cos(theta3)]])
zxr = Matrix([[d2 - sin(theta3)], [0], [0]])
col = Matrix([[0], [0], [1], [0], [0], [0]])
____________________________________________________________________________________________________
i =  1 __________________________________________________________________________________________
T   = Matrix([[1, 0, 0, 0], [0, 0, -1, 0], [0, 1, 0, d1], [0, 0, 0, 1]])
Z   = Matrix([[0], [-1], [0]])
P   = Matrix([[0], [0], [d1]])
r   = Matrix([[0], [-d2 + sin(theta3)], [cos(theta3)]])
zxr = Matrix([[-cos(theta3)], [0], [0]])
col = Matrix([[0], [-1], [0], [0], [0], [0]])
____________________________________________________________________________________________________
i =  2 ______________

In [ ]:
print('Inverse Kinematics')
t1 = tMatrix("theta1", 1, 90 ,0)
t2 = tMatrix("theta2", 0, 0 , 1)
t3 = tMatrix("theta3", 0, 0, 0)
t4 = tMatrix(0, 0, 0, 0.5)
fk: tMatrix = t1*t2*t3*t4

target = [
[0.612,     -0.354,     0.707 ,     0.806],     
[0.612,     -0.354,     -0.707,     0.806],     
[0.500,     0.866 ,     0.0   ,     1.957],     
[0    ,     0     ,     0     ,     1    ]]

ik = fk.ik(target)
for i in ik:
    print(f'{i} = {ik[i] * 180 / pi}')


Inverse Kinematics
theta1 = 44.9994669103490
theta2 = 45.0004007519710
theta3 = -14.9783222213111
